<a href="https://colab.research.google.com/github/lucascosta7/analise-combustiveis/blob/main/analise_combustiveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Analise-Combustiveis")
    .getOrCreate()
)

In [3]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv("/content/data/precos-gasolina-etanol-10.csv")
    )

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
df_precos = (
    df
    .select('Estado - Sigla', 'Municipio', 'Produto', 'Valor de Compra', 'Valor de Venda', 'Unidade de Medida', 'Bandeira')
)

In [6]:
(
    df_precos
    .where(
        F.col('Valor de Compra').isNotNull()
    )
    .show()
)

+--------------+---------+-------+---------------+--------------+-----------------+--------+
|Estado - Sigla|Municipio|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|Bandeira|
+--------------+---------+-------+---------------+--------------+-----------------+--------+
+--------------+---------+-------+---------------+--------------+-----------------+--------+



In [7]:
df_precos = (
    df
    .select('Estado - Sigla', 'Municipio', 'Produto', 'Valor de Venda', 'Unidade de Medida', 'Bandeira')
    .withColumn(
        "Valor de Venda",
        F.regexp_replace(F.col("Valor de Venda"), ",", ".")
        .cast("float")
    )
)

In [8]:
df_produtos_analise = (
    df_precos
    .groupBy(
        F.col('Produto')
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias('menor_preco'),
        F.max(F.col("Valor de Venda")).alias('maior_preco')
    )
    .show()
)

+------------------+-----------+-----------+
|           Produto|menor_preco|maior_preco|
+------------------+-----------+-----------+
|GASOLINA ADITIVADA|       4.79|       8.69|
|            ETANOL|       2.78|        6.6|
|          GASOLINA|       4.59|       7.62|
+------------------+-----------+-----------+



In [9]:
df_precos_media = (
    df_precos
    .groupBy(
        F.col('Estado - Sigla')
    )
    .avg("Valor de Venda")
    .show()
)

+--------------+-------------------+
|Estado - Sigla|avg(Valor de Venda)|
+--------------+-------------------+
|            SC| 5.5442990016654505|
|            RO|  6.239915405634898|
|            PI|  5.207308851615369|
|            AM|  6.081273168964959|
|            RR|  5.864293734232585|
|            GO|   4.90600106857016|
|            TO|  5.541441934832027|
|            MT|  4.819765392286687|
|            SP|  4.913365221043784|
|            ES|  5.551594053605111|
|            PB|  5.276236401627893|
|            RS|   5.58953165012815|
|            MS|  5.193745138102057|
|            AL|  5.478569420178731|
|            MG|  4.929369353885123|
|            PA|   6.01306065762368|
|            BA|  5.464833559839801|
|            SE|    5.7051443477941|
|            PE| 5.2422393091248525|
|            CE|  5.535351578885313|
+--------------+-------------------+
only showing top 20 rows



In [10]:
df_precos_bandeira = (
    df_precos
    .groupBy(
        F.col('Bandeira')
    )
    .avg("Valor de Venda")
    .show()
)

+-------------------+-------------------+
|           Bandeira|avg(Valor de Venda)|
+-------------------+-------------------+
|              SABBÁ|  5.758253679636872|
|              VIBRA|  5.356094067805941|
|             RODOIL| 5.4788188709048775|
|             BRANCA| 5.0304336267132514|
|      TOTALENERGIES| 4.8968235100016875|
|         RIO BRANCO|   5.22028568812779|
|             ALESAT|  5.156823870289942|
|   SUL COMBUSTÍVEIS|  5.824545600197532|
|             TAURUS|  4.936012616640405|
|          POTENCIAL|  5.413124990463257|
|                FAN|  5.617543781012819|
|              LARCO|  5.160952443168277|
|         PETROBAHIA|  5.481153833560454|
|                 SP| 5.5932393879957605|
|              IDAZA|  5.559411736095653|
|MASUT DISTRIBUIDORA| 4.9091176355586335|
|          SIMARELLI|  4.810909054496071|
|             TEMAPE|  5.262380951926822|
|           ON PETRO|  5.485833326975505|
|        RAIZEN MIME|    5.5468292294479|
+-------------------+-------------

In [11]:
df_precos_filtro = (
    df_precos
    .filter(F.col('Estado - Sigla') == 'SP')
    .groupBy(F.col('Produto'))
    .avg("Valor de Venda")
    .show()
)

+------------------+-------------------+
|           Produto|avg(Valor de Venda)|
+------------------+-------------------+
|GASOLINA ADITIVADA|  5.843014348713785|
|            ETANOL| 3.4604334426492116|
|          GASOLINA|  5.590069530250139|
+------------------+-------------------+



In [12]:
df_municipios = (
    df
    .select('Municipio')
)

In [13]:
df_municipios_analise = (
    df_municipios
    .groupBy('Municipio')
    .count()
    .orderBy(F.col("count"))
    .show(10)
)

+--------------------+-----+
|           Municipio|count|
+--------------------+-----+
|            ITAITUBA|    7|
|               LAGES|   10|
|           VERA CRUZ|   10|
|SANTA CRUZ DO RIO...|   11|
|              MARABA|   11|
|              TOLEDO|   12|
|        POUSO ALEGRE|   15|
|                TEFE|   18|
|          JAGUAQUARA|   21|
|            REDENCAO|   22|
+--------------------+-----+
only showing top 10 rows

